In [2]:
import pandas as pd 
import csv
from datetime import datetime
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF8')

'en_US.UTF8'

In [2]:
ada_lead_path = 'data/raw/ada_lead_data.csv'
qq_export_path = 'data/QQ_V4.csv'
ws_path = 'data/raw/Waterstone_Book.csv'
ag_path = 'data/raw/A_Gange_Book.csv'

### Ada

In [3]:
df_ada = pd.read_csv(ada_lead_path, index_col=0)
df_ada = df_ada.dropna(subset=['email'])
customers_ada_set = set()
for index, row in df_ada.iterrows():
#     if row['state'] != '1':
#         continue
    full_name = row['name'] if isinstance(row['name'], str) else '' 
    email = row['email'].lower() if isinstance(row['email'], str) else ''
    policy_num = row['policy_number'].upper() if isinstance(row['policy_number'], str) else ''
    first_name, last_name = full_name.split(' ')[0].capitalize(), full_name.split(' ')[-1].capitalize()
    customers_ada_set.add((first_name, last_name, email, policy_num))
len(customers_ada_set)

1158

#### WaterStone and A Gange

In [4]:
df_ag = pd.read_csv(ag_path)#.dropna(subset=['Policy Number'])
print(len(df_ag))
df_ws = pd.read_csv(ws_path)#.dropna(subset=['Policy Number'])
print(len(df_ws))

6345
29387


In [5]:
overlap_set = set()
df_overlap = pd.DataFrame(columns= df_ag.columns)
df_null = pd.DataFrame(columns= df_ag.columns)

In [6]:
# def extract_cust(df, overlap, ada_set):
#     #df = df.dropna(subset=['Customer Email'])
#     df_null = pd.DataFrame(columns= df.columns)
#     df_overlap = pd.DataFrame(columns= df.columns)
#     df_valid = pd.DataFrame(columns= df.columns)
#     cust = set()
#     for index, row in df.iterrows():
#         first_name = row['Customer First Name'].capitalize() if isinstance(row['Customer First Name'], str) else ''
#         last_name = row['Customer Last Name'].capitalize() if isinstance(row['Customer Last Name'], str) else ''
#         zip_code = zip_fomatter(row['Customer Zip Code']) if isinstance(row['Customer Zip Code'], str) else ''
#         email =  row['Customer Email'].lower() if isinstance(row['Customer Email'], str) else ''
#         policy_num = row['Policy Number'].upper() if isinstance(row['Policy Number'], str) else ''
        
#         if not policy_num or not first_name or not last_name:
#             df_null = df_null.append(df.loc[index],  ignore_index = True)
#             continue
#         if (first_name, last_name, email, policy_num) in ada_set:
#             overlap.add((first_name, last_name, email, policy_num))
#             df_overlap = df_overlap.append(df.loc[index],  ignore_index = True)
#         else:
#             customer_info = (first_name, last_name, zip_code, email, policy_num)
#             if customer_info in cust:
#                 continue
#             cust.add(customer_info)
#             df_valid = df_valid.append(df.loc[index],  ignore_index = True)
#     return cust, df_null, df_overlap, df_valid

# def zip_fomatter(zipcode):
#     if not zipcode:
#         return ''
#     main_part = zipcode.split('-')[0]
#     if len(main_part) == 4:
#         return '0'+zipcode
#     return zipcode

def extract_cust(df, overlap, ada_set):
    #df = df.dropna(subset=['Customer Email'])
    df_null = pd.DataFrame(columns= df.columns)
    df_overlap = pd.DataFrame(columns= df.columns)
    df_valid = pd.DataFrame(columns= df.columns)
    cust = set()
    for index, row in df.iterrows():
        first_name = row['Customer First Name'].capitalize() if isinstance(row['Customer First Name'], str) else ''
        last_name = row['Customer Last Name'].capitalize() if isinstance(row['Customer Last Name'], str) else ''
        email =  row['Customer Email'].lower() if isinstance(row['Customer Email'], str) else ''
        policy_num = row['Policy Number'].upper() if isinstance(row['Policy Number'], str) else ''
        
        if not policy_num or not first_name or not last_name:
            df_null = df_null.append(df.loc[index],  ignore_index = True)
            continue
        if (first_name, last_name, email, policy_num) in ada_set:
            overlap.add((first_name, last_name, email, policy_num))
            df_overlap = df_overlap.append(df.loc[index],  ignore_index = True)
        else:
            customer_info = (first_name, last_name, email, policy_num)
            if customer_info in cust:
                continue
            cust.add(customer_info)
            df_valid = df_valid.append(df.loc[index],  ignore_index = True)
    return cust, df_null, df_overlap, df_valid

def zip_fomatter(zipcode):
    if not zipcode:
        return ''
    main_part = zipcode.split('-')[0]
    if len(main_part) == 4:
        return '0'+zipcode
    return zipcode

In [7]:
customers_ws_set, df_ws_null, df_ws_overlap, df_ws_valid = extract_cust(df_ws, overlap_set, customers_ada_set)
customers_ag_set, df_ag_null, df_ag_overlap, df_ag_valid = extract_cust(df_ag, overlap_set, customers_ada_set)

In [33]:
len(df_ws_valid), len(df_ag_valid)

(7715, 2383)

### QQ

In [7]:
df_qq = pd.read_csv(qq_export_path)
len(df_qq)

18845

In [8]:
customers_qq_set, df_qq_null, df_qq_overlap, df_qq_valid = extract_cust(df_qq, overlap_set, customers_ada_set)

In [9]:
df_qq_valid

,Customer First Name,Customer Last Name,CSR On Customer,Agent On Policy,CSR On Policy,Customer Email,Producer,Policy Number,Policy Premium,Agent On Customer,Customer Number,Effective Date,Expiration Date,Parent Carrier,Customer Primary Email,Writing Carrier
0,Tony,Gross,Linda Sabino,Linda Sabino,Linda Sabino,rmgross@verizon.net,Linda Sabino,KKO0000008839000,"$1,399.00",Linda Sabino,WSDC-03825,6/26/2022,6/26/2023,Nationwide Corp Grp,tonygross1932@gmail.com,National Cas Co
1,Tony,Gross,Linda Sabino,Linda Sabino,Linda Sabino,tonygross1932@gmail.com,Linda Sabino,KKO0000008839000,"$1,399.00",Linda Sabino,WSDC-03825,6/26/2022,6/26/2023,Nationwide Corp Grp,tonygross1932@gmail.com,National Cas Co
2,MARGO,GRISELL,Richard McLaughlin,Franc Kalanderi,Franc Kalanderi,NaN,NaN,6081778336331,"$1,340.00",Richard McLaughlin,WSDC-04433,12/30/2021,12/30/2022,Travelers Grp,NaN,Travelers Personal Ins Co
3,MICHAEL,METZERMACHER,Aaron Culp,Aaron Culp,Aaron Culp,NaN,NaN,K3788820,"$1,064.20",Aaron Culp,NaN,12/30/2021,6/30/2022,Liberty Mut Grp,NaN,Safeco Ins Co Of Il
4,Ashley,Zapotosky,NaN,Linda Sabino,Linda Sabino,ashzap12@gmail.com,Linda Sabino,6BRPG0000007214500,$0.00,Linda Sabino,WSDC-03663,12/27/2021,12/27/2022,Nationwide Corp Grp,ashzap12@gmail.com,National Cas Co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10777,Andrew,Napolitano,Linda Sabino,Linda Sabino,NaN,horshamquestions@hullco.com,NaN,Pending,$0.00,Linda Sabino,WSDC-03358,1/1/0001,1/1/0001,NaN,angnap67@gmail.com,NaN
10778,Andrew,Napolitano,Linda Sabino,Linda Sabino,NaN,josie@forlenzacpa.com,NaN,Pending,$0.00,Linda Sabino,WSDC-03358,1/1/0001,1/1/0001,NaN,angnap67@gmail.com,NaN
10779,Andrew,Napolitano,Linda Sabino,Linda Sabino,NaN,mschultz@hullco.com,NaN,Pending,$0.00,Linda Sabino,WSDC-03358,1/1/0001,1/1/0001,NaN,angnap67@gmail.com,NaN
10780,Andrew,Napolitano,Linda Sabino,Linda Sabino,NaN,pdriscoll@hullco.com,NaN,Pending,$0.00,Linda Sabino,WSDC-03358,1/1/0001,1/1/0001,NaN,angnap67@gmail.com,NaN


In [10]:
# df_other = pd.concat([df_qq_null, df_qq_overlap])
# df_other.to_csv('data/qq_null_and_inada.csv')

In [11]:
not_wts_agents = {"Albert Meng","Aaron Culp","Michael Giambrone",\
                  "Michael Coughlin","Richard McLaughlin","Tony Lin",\
                  "Xavier Ke","Franc Kalanderi","Gaurav Garg",\
                  "Hunter Brown","Joshua Grant","Karin D'Eramo"}

In [12]:
df_after_filtered = df_qq_valid[(~df_qq_valid['CSR On Customer'].isin(not_wts_agents))\
                                &(~df_qq_valid['CSR On Policy'].isin(not_wts_agents))\
                               &(~df_qq_valid['Agent On Policy'].isin(not_wts_agents))\
                               &(~df_qq_valid['Producer'].isin(not_wts_agents))]

In [13]:
df_after_filtered_left = df_qq_valid[(df_qq_valid['CSR On Customer'].isin(not_wts_agents))\
                                |(df_qq_valid['CSR On Policy'].isin(not_wts_agents))\
                               |(df_qq_valid['Agent On Policy'].isin(not_wts_agents))\
                               |(df_qq_valid['Producer'].isin(not_wts_agents))]

In [40]:
df_after_filtered.to_csv('data/Output/qq_valid_filtered.csv')

In [41]:
df_after_filtered_left.to_csv('data/Output/qq_not_waterstone.csv')

### Further Processing

In [14]:
def get_customer_info_from_df(df, ada_set, not_waterstone_set = set()):
    df_result = pd.DataFrame(columns= df.columns)
    customers = set()
    overlap_set = set()
    
    for index, row in df.iterrows():
        first_name = row['Customer First Name'].capitalize() if isinstance(row['Customer First Name'], str) else ''
        last_name = row['Customer Last Name'].capitalize() if isinstance(row['Customer Last Name'], str) else ''
        email =  row['Customer Email'].lower() if isinstance(row['Customer Email'], str) else ''
        cust_num =  row['Customer Number'].lower() if isinstance(row['Customer Number'], str) else ''
        if (first_name, last_name, email) not in ada_set:
            customer_info = (first_name, last_name, cust_num)
            if customer_info in customers:
                continue
            if customer_info in not_waterstone_set and customer_info not in overlap_set:
                print(customer_info)
                overlap_set.add(customer_info)
                continue
            customers.add(customer_info)
            df_result = df_result.append(df.loc[index],  ignore_index = True)
            
    return customers, df_result, overlap_set

def get_policy_based_info_from_df(df, ada_set, not_waterstone_set = set()):
    df_result = pd.DataFrame(columns= df.columns)
    customers = set()
    overlap_set = set()
    
    for index, row in df.iterrows():
        first_name = row['Customer First Name'].capitalize() if isinstance(row['Customer First Name'], str) else ''
        last_name = row['Customer Last Name'].capitalize() if isinstance(row['Customer Last Name'], str) else ''
        email =  row['Customer Email'].lower() if isinstance(row['Customer Email'], str) else ''
        policy_num =  row['Policy Number'].lower() if isinstance(row['Policy Number'], str) else ''
        
        if (first_name, last_name, email) not in ada_set:
            customer_info = (first_name, last_name, cust_num)
            if customer_info in not_waterstone_set and customer_info not in overlap_set:
                print(customer_info)
                overlap_set.add(customer_info)
                continue
            customers.add(customer_info)
            df_result = df_result.append(df.loc[index],  ignore_index = True)
            
    return customers, df_result, overlap_set

In [19]:
cust_set_not_in_waterstone, _, _ = get_customer_info_from_df(df_after_filtered_left, customers_ada_set)
len(cust_set_not_in_waterstone)

2738

In [25]:
cust_set_from_waterstone, df_result, overlap_set = get_customer_info_from_df(df_after_filtered, customers_ada_set, cust_set_not_in_waterstone)

('Matthew', 'Dvorozniak', 'wsdc-00239')
('Tara', 'Cummings', 'wsdc-03863')
('Michael', 'Zangoglia', 'wsdc-03364')


In [15]:
df_overlap = pd.DataFrame(columns= df_after_filtered.columns)
df_final_result = pd.DataFrame(columns= df_after_filtered.columns)
for index, row in df_after_filtered.iterrows():
    first_name = row['Customer First Name'].capitalize() if isinstance(row['Customer First Name'], str) else ''
    last_name = row['Customer Last Name'].capitalize() if isinstance(row['Customer Last Name'], str) else ''
    email =  row['Customer Email'].lower() if isinstance(row['Customer Email'], str) else ''
    cust_num =  row['Customer Number'].lower() if isinstance(row['Customer Number'], str) else ''
    
    customer_info = (first_name, last_name, cust_num)
    if customer_info in overlap_set:
        df_overlap = df_overlap.append(df_after_filtered.loc[index],  ignore_index = True)
    else:
        df_final_result = df_final_result.append(df_after_filtered.loc[index],  ignore_index = True)
df_overlap

,Customer First Name,Customer Last Name,CSR On Customer,Agent On Policy,CSR On Policy,Customer Email,Producer,Policy Number,Policy Premium,Agent On Customer,Customer Number,Effective Date,Expiration Date,Parent Carrier,Customer Primary Email,Writing Carrier


In [17]:
df_final_result

,Customer First Name,Customer Last Name,CSR On Customer,Agent On Policy,CSR On Policy,Customer Email,Producer,Policy Number,Policy Premium,Agent On Customer,Customer Number,Effective Date,Expiration Date,Parent Carrier,Customer Primary Email,Writing Carrier
0,Tony,Gross,Linda Sabino,Linda Sabino,Linda Sabino,rmgross@verizon.net,Linda Sabino,KKO0000008839000,"$1,399.00",Linda Sabino,WSDC-03825,6/26/2022,6/26/2023,Nationwide Corp Grp,tonygross1932@gmail.com,National Cas Co
1,Tony,Gross,Linda Sabino,Linda Sabino,Linda Sabino,tonygross1932@gmail.com,Linda Sabino,KKO0000008839000,"$1,399.00",Linda Sabino,WSDC-03825,6/26/2022,6/26/2023,Nationwide Corp Grp,tonygross1932@gmail.com,National Cas Co
2,Ashley,Zapotosky,NaN,Linda Sabino,Linda Sabino,ashzap12@gmail.com,Linda Sabino,6BRPG0000007214500,$0.00,Linda Sabino,WSDC-03663,12/27/2021,12/27/2022,Nationwide Corp Grp,ashzap12@gmail.com,National Cas Co
3,Ashley,Zapotosky,NaN,Linda Sabino,Linda Sabino,info@fitnessinsurance-kk.com,Linda Sabino,6BRPG0000007214500,$0.00,Linda Sabino,WSDC-03663,12/27/2021,12/27/2022,Nationwide Corp Grp,ashzap12@gmail.com,National Cas Co
4,Ashley,Zapotosky,NaN,Linda Sabino,Linda Sabino,tiffany.a.haugh@kandkinsurance.com,Linda Sabino,6BRPG0000007214500,$0.00,Linda Sabino,WSDC-03663,12/27/2021,12/27/2022,Nationwide Corp Grp,ashzap12@gmail.com,National Cas Co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6469,Andrew,Napolitano,Linda Sabino,Linda Sabino,NaN,horshamquestions@hullco.com,NaN,Pending,$0.00,Linda Sabino,WSDC-03358,1/1/0001,1/1/0001,NaN,angnap67@gmail.com,NaN
6470,Andrew,Napolitano,Linda Sabino,Linda Sabino,NaN,josie@forlenzacpa.com,NaN,Pending,$0.00,Linda Sabino,WSDC-03358,1/1/0001,1/1/0001,NaN,angnap67@gmail.com,NaN
6471,Andrew,Napolitano,Linda Sabino,Linda Sabino,NaN,mschultz@hullco.com,NaN,Pending,$0.00,Linda Sabino,WSDC-03358,1/1/0001,1/1/0001,NaN,angnap67@gmail.com,NaN
6472,Andrew,Napolitano,Linda Sabino,Linda Sabino,NaN,pdriscoll@hullco.com,NaN,Pending,$0.00,Linda Sabino,WSDC-03358,1/1/0001,1/1/0001,NaN,angnap67@gmail.com,NaN


In [16]:
df_final_result = df_final_result[(df_final_result['Customer State/Province'] == 'NJ') | (df_final_result['Customer State/Province'] == 'NY')]
df_final_result

KeyError: 'Customer State/Province'

In [23]:
_, final_result, _ = get_customer_info_from_df(df_final_result, customers_ada_set)

In [24]:
df_overlap = pd.DataFrame(columns= df_after_filtered_left.columns)
for index, row in df_after_filtered_left.iterrows():
    first_name = row['Customer First Name'].capitalize() if isinstance(row['Customer First Name'], str) else ''
    last_name = row['Customer Last Name'].capitalize() if isinstance(row['Customer Last Name'], str) else ''
    email =  row['Customer Email'].lower() if isinstance(row['Customer Email'], str) else ''
    cust_num =  row['Customer Number'].lower() if isinstance(row['Customer Number'], str) else ''
    
    customer_info = (first_name, last_name, cust_num)
    if customer_info in overlap_set:
        df_overlap = df_overlap.append(df_after_filtered_left.loc[index],  ignore_index = True)
df_overlap

,Customer First Name,Customer Last Name,Customer Street Address,Customer State/Province,Customer City,Customer Zip Code,Customer Email,Customer Phone Number,Customer Number,CSR On Customer,Policy Number,Policy Premium,Policy Status,Policy Type,Agent On Policy,CSR On Policy,Effective Date,Producer,Parent Carrier
0,TARA,CUMMINGS,19 COTTAGE AVE,NJ,MONTVALE,07645-2506,NaN,(201) 661-0667,WSDC-03863,NaN,6070118562061,"$1,974.00",Active,Single,Richard McLaughlin,Richard McLaughlin,7/7/2021,NaN,Travelers Grp
1,Michael,Zangoglia,92 CEDAR LN # 4D,NJ,TEANECK,07666,59.mzbuilder1@gmail.com,(201) 290-2876,WSDC-03364,Christie Murphy,6083954146341,$126.00,Cancelled,Single,Richard McLaughlin,Richard McLaughlin,2/3/2021,NaN,Travelers Grp
2,Michael,Zangoglia,92 CEDAR LN # 4D,NJ,TEANECK,07666,59.mzbuilder1@gmail.com,(201) 290-2876,WSDC-03364,Christie Murphy,6083954152061,"$1,462.00",Active,Single,Richard McLaughlin,Richard McLaughlin,2/3/2021,NaN,Travelers Grp
3,Matthew,Dvorozniak,62 Engle St,NJ,Cresskill,07626,dvorozniak@hotmail.com,(201) 232-9078,WSDC-00239,NaN,CP9022147,$288.00,Active,Single,Richard McLaughlin,Richard McLaughlin,1/14/2021,NaN,Wbl Grp
4,Matthew,Dvorozniak,62 Engle St,NJ,Cresskill,07626,mardvor@hotmail.com,(201) 232-9078,WSDC-00239,NaN,CP9022147,$288.00,Active,Single,Richard McLaughlin,Richard McLaughlin,1/14/2021,NaN,Wbl Grp


In [97]:
df_overlap.to_csv('overlap_in_customer_in_waterstone.csv')

In [107]:
final_result.to_csv('data/final_result.csv')

In [26]:
final_result

,Customer First Name,Customer Last Name,Customer Street Address,Customer State/Province,Customer City,Customer Zip Code,Customer Email,Customer Phone Number,Customer Number,CSR On Customer,Policy Number,Policy Premium,Policy Status,Policy Type,Agent On Policy,CSR On Policy,Effective Date,Producer,Parent Carrier
0,Tony,Gross,P.O. Box 187,NJ,Lake Hopatcong,07849,rmgross@verizon.net,(973) 584-6311,WSDC-03825,Linda Sabino,KKO0000008839000,"$1,399.00",Cancelled,Package,Linda Sabino,Linda Sabino,6/26/2022,Linda Sabino,Nationwide Corp Grp
1,Ashley,Zapotosky,122 Maple Ave.,NJ,Wallington,07057,ashzap12@gmail.com,(201) 446-1709,WSDC-03663,NaN,6BRPG0000007214500,$0.00,Pending,Single,Linda Sabino,Linda Sabino,12/27/2021,Linda Sabino,Nationwide Corp Grp
2,Maria,Valdivia,309 Valley Rd,NJ,Haworth,07641-1220,valdivia.octavio@gmail.com,(201) 400-7122,WSDC-00810,NaN,4143998,"$2,686.00",Active,Single,WATERSTONE HOUSE AGENT,Christie Murphy,12/10/2021,NaN,Franklin Mut Grp
3,Manfred,Krick,50 Huntting Dr,NJ,Dumont,07628-1517,mfkrick@optonline.net,(201) 387-2073,WSDC-00446,NaN,4144309,"$1,069.00",Active,Single,WATERSTONE HOUSE AGENT,Christie Murphy,12/9/2021,NaN,Franklin Mut Grp
4,Lunier,Dominguez-Leon,2020 Dill Ave,NJ,Linden,07036-1010,lunierdl@gmail.com,(908) 248-3659,WSDC-02357,WATERSTONE HOUSE AGENT,NJAP0000031665,$870.19,Active,Single,WATERSTONE HOUSE AGENT,Christie Murphy,12/7/2021,NaN,Mercury Gen Grp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976,tbd,tbd,32 First St,NJ,Dumont,07628,NaN,NaN,WSDC-03103,WATERSTONE HOUSE AGENT,SCP1005853,$507.00,Expired,Single,OLD MCFADDEN,WATERSTONE HOUSE AGENT,10/16/2016,OLD MCFADDEN,Franklin Mut Grp
1977,Dollar,LLC,445 Old Hook Road,NJ,Emerson,07630,NaN,NaN,WSDC-00228,NaN,SBP2590370,"$1,521.00",Cancelled,Single,WATERSTONE HOUSE AGENT,WATERSTONE HOUSE AGENT,7/27/2016,WATERSTONE HOUSE AGENT,Franklin Mut Grp
1978,Sga,LLC,622 Cleveland Avenue,NJ,River Vale,07675,sarcid@aol.com,(201) 841-3569,WSDC-00736,NaN,08140671,"$6,408.00",Cancelled,Single,WATERSTONE HOUSE AGENT,WATERSTONE HOUSE AGENT,7/14/2016,WATERSTONE HOUSE AGENT,Progressive Grp
1979,Michael,LaGratta,pob 104,NJ,oradell,,a98beemer@aol.com,NaN,WSDC-03108,WATERSTONE HOUSE AGENT,tbd,$327.28,Expired,Single,WATERSTONE HOUSE AGENT,WATERSTONE HOUSE AGENT,6/12/2016,NaN,Berkshire Hathaway Grp


### Premium Calculation

In [27]:
_, df_policies_of_waterstone, _ = get_policy_based_info_from_df(df_after_filtered, customers_ada_set)

In [28]:
df_policies_of_waterstone

,Customer First Name,Customer Last Name,Customer Street Address,Customer State/Province,Customer City,Customer Zip Code,Customer Email,Customer Phone Number,Customer Number,CSR On Customer,Policy Number,Policy Premium,Policy Status,Policy Type,Agent On Policy,CSR On Policy,Effective Date,Producer,Parent Carrier
0,Tony,Gross,P.O. Box 187,NJ,Lake Hopatcong,07849,rmgross@verizon.net,(973) 584-6311,WSDC-03825,Linda Sabino,KKO0000008839000,"$1,399.00",Cancelled,Package,Linda Sabino,Linda Sabino,6/26/2022,Linda Sabino,Nationwide Corp Grp
1,Tony,Gross,P.O. Box 187,NJ,Lake Hopatcong,07849,tonygross1932@gmail.com,(973) 584-6311,WSDC-03825,Linda Sabino,KKO0000008839000,"$1,399.00",Cancelled,Package,Linda Sabino,Linda Sabino,6/26/2022,Linda Sabino,Nationwide Corp Grp
2,Ashley,Zapotosky,122 Maple Ave.,NJ,Wallington,07057,ashzap12@gmail.com,(201) 446-1709,WSDC-03663,NaN,6BRPG0000007214500,$0.00,Pending,Single,Linda Sabino,Linda Sabino,12/27/2021,Linda Sabino,Nationwide Corp Grp
3,Ashley,Zapotosky,122 Maple Ave.,NJ,Wallington,07057,info@fitnessinsurance-kk.com,(201) 446-1709,WSDC-03663,NaN,6BRPG0000007214500,$0.00,Pending,Single,Linda Sabino,Linda Sabino,12/27/2021,Linda Sabino,Nationwide Corp Grp
4,Ashley,Zapotosky,122 Maple Ave.,NJ,Wallington,07057,tiffany.a.haugh@kandkinsurance.com,(201) 446-1709,WSDC-03663,NaN,6BRPG0000007214500,$0.00,Pending,Single,Linda Sabino,Linda Sabino,12/27/2021,Linda Sabino,Nationwide Corp Grp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6457,Andrew,Napolitano,873 Broadway,NJ,Newark,07104,horshamquestions@hullco.com,(201) 303-8552,WSDC-03358,Linda Sabino,Pending,$0.00,Pending,Single,Linda Sabino,NaN,1/1/0001,NaN,NaN
6458,Andrew,Napolitano,873 Broadway,NJ,Newark,07104,josie@forlenzacpa.com,(201) 303-8552,WSDC-03358,Linda Sabino,Pending,$0.00,Pending,Single,Linda Sabino,NaN,1/1/0001,NaN,NaN
6459,Andrew,Napolitano,873 Broadway,NJ,Newark,07104,mschultz@hullco.com,(201) 303-8552,WSDC-03358,Linda Sabino,Pending,$0.00,Pending,Single,Linda Sabino,NaN,1/1/0001,NaN,NaN
6460,Andrew,Napolitano,873 Broadway,NJ,Newark,07104,pdriscoll@hullco.com,(201) 303-8552,WSDC-03358,Linda Sabino,Pending,$0.00,Pending,Single,Linda Sabino,NaN,1/1/0001,NaN,NaN


In [29]:
# change Effective Date from string Type to datetime Type
df_policies_of_waterstone['Effective Date'] = df_policies_of_waterstone['Effective Date'].apply(lambda x: datetime. strptime(x, '%m/%d/%Y'))


In [30]:
# change Premium from string Type to Float Type 
df_policies_of_waterstone['Policy Premium'] = df_policies_of_waterstone['Policy Premium'].apply(lambda x: locale.atof(x.strip("$")))

In [31]:
# filter by date in 2021
waterstone_policies_2021 = df_policies_of_waterstone.loc[(df_policies_of_waterstone['Effective Date'] >= datetime. strptime('12/31/2020', '%m/%d/%Y'))
                     & (df_policies_of_waterstone['Effective Date'] < datetime. strptime('01/01/2022', '%m/%d/%Y'))]

# filter by date in 2020
waterstone_policies_2020 = df_policies_of_waterstone.loc[(df_policies_of_waterstone['Effective Date'] >= datetime. strptime('12/31/2019', '%m/%d/%Y'))
                     & (df_policies_of_waterstone['Effective Date'] < datetime. strptime('01/01/2021', '%m/%d/%Y'))]



In [184]:
filtered_df = filtered_df[(filtered_df['CSR On Customer'] == 'Linda Sabino')\
                                |(filtered_df['CSR On Policy'] == 'Linda Sabino')\
                               |(filtered_df['Agent On Policy'] == 'Linda Sabino')\
                               |(filtered_df['Producer'] == 'Linda Sabino')]

In [32]:
waterstone_policies_2021.groupby(['Policy Status'])['Policy Premium'].sum()

Policy Status
Active       5340644.60
Cancelled     173038.74
Expired         6765.00
Pending        47488.00
Voided             0.00
Name: Policy Premium, dtype: float64

In [33]:
waterstone_policies_2020.groupby(['Policy Status'])['Policy Premium'].sum()

Policy Status
Active       3070627.40
Cancelled     178049.37
Expired      3847296.53
Pending        20534.00
Voided         27107.00
Name: Policy Premium, dtype: float64

In [34]:
waterstone_policies_2021.groupby(['Parent Carrier', 'Policy Status'])['Policy Premium'].sum()

Parent Carrier                Policy Status
Ace Ltd Grp                   Active            15788.55
Allianz Ins Grp               Active           113287.44
                              Cancelled         37396.35
BERKSHIRE HATHAWAY INC        Active           164778.00
                              Cancelled         40165.00
Berkshire Hathaway Grp        Active           103127.00
                              Pending               0.00
Cna Ins Grp                   Active            15750.00
Cumberland Grp                Active            60434.00
Employers Holdings Grp        Active            19573.00
Fairfax Fin Grp               Active           534585.95
Farmers Ins Grp               Active              842.00
Franklin Mut Grp              Active           228278.00
                              Cancelled         14686.00
Ironshore Grp                 Active             1397.82
Liberty Mut Grp               Active              950.00
Meadowbrook Ins Grp           Active        

In [35]:
waterstone_policies_2021[waterstone_policies_2021['Parent Carrier'].isnull()].groupby(['Policy Status']).sum()

,Policy Premium
Policy Status,
Active,2392121.70
Cancelled,37413.21
Expired,3810.00
Pending,0.00
Voided,0.00


In [70]:
waterstone_policies_2021.to_csv('waterstone_policies_2021.csv')

In [40]:
new_list = ['Original CSR', 'If Exist']
#tmp_2020_policies = waterstone_policies_2020.reindex(columns=[*waterstone_policies_2020.columns.to_list(), 'Original Agent', 'Original CSR', 'If Exist'])
#waterstone_policies_2020.insert(1, "Original CSR", '', True)

_, tmp_2020_customer, _ = get_customer_info_from_df(waterstone_policies_2020, customers_ada_set)
tmp_2020_customer = tmp_2020_customer.reindex(columns=[*waterstone_policies_2020.columns.to_list(), 'Original Agent', 'Original CSR', 'If Exist'])

In [43]:
tmp_2020_customer.to_csv('tmp_2020_customer.csv')

In [44]:
input_file = csv.DictReader(open("tmp_2020_customer.csv"))

In [45]:
input_file

In [22]:
qq_writting = pd.read_csv('data/QQ_1028_writting.csv')
qq_parent = pd.read_csv('data/QQ_1028_parent.csv')
qq_MGA = pd.read_csv('data/QQ_1028_parent.csv')

In [37]:
list(qq_writting['Customer Number']) == list(qq_parent['Customer Number']) 

True

In [42]:
cnt = 0
for i in range(len(qq_writting['Writing Carrier'])):
    w = list(qq_writting['Writing Carrier'])[i]
    p = list(qq_parent['Parent Carrier'])[i]
    if w != p:
        print(w + ' ---- '+ p)
        cnt+=1
        if cnt == 10:
            break

National Cas Co ---- Nationwide Corp Grp
National Cas Co ---- Nationwide Corp Grp
Safeco Ins Co Of Il ---- Liberty Mut Grp
Travelers Personal Ins Co ---- Travelers Grp
National Cas Co ---- Nationwide Corp Grp
National Cas Co ---- Nationwide Corp Grp
National Cas Co ---- Nationwide Corp Grp
Liberty Cnty Mut Ins Co ---- Liberty Mut Grp
American Economy Ins Co ---- Liberty Mut Grp
American Economy Ins Co ---- Liberty Mut Grp


In [29]:
qq_parent

,Customer First Name,Customer Last Name,Customer Street Address,Customer State/Province,Customer Zip Code,Customer Email,Customer Phone Number,Customer Number,Policy Number,Policy Premium,Policy Status,CSR On Customer,Agent On Policy,CSR On Policy,Producer,Effective Date,Parent Carrier
0,Tony,Gross,P.O. Box 187,NJ,07849,rmgross@verizon.net,(973) 584-6311,WSDC-03825,KKO0000008839000,"$1,399.00",Cancelled,Linda Sabino,Linda Sabino,Linda Sabino,Linda Sabino,6/26/2022,Nationwide Corp Grp
1,Tony,Gross,P.O. Box 187,NJ,07849,tonygross1932@gmail.com,(973) 584-6311,WSDC-03825,KKO0000008839000,"$1,399.00",Cancelled,Linda Sabino,Linda Sabino,Linda Sabino,Linda Sabino,6/26/2022,Nationwide Corp Grp
2,MICHAEL,METZERMACHER,29 BOSWELL AVE,CT,06360-7901,NaN,(860) 912-4182,NaN,K3788820,"$1,064.20",Active,Aaron Culp,Aaron Culp,Aaron Culp,NaN,12/30/2021,Liberty Mut Grp
3,MARGO,GRISELL,54 HOP BROOK RD,CT,06804,NaN,(203) 885-2228,WSDC-04433,6081778336331,"$1,340.00",Active,Richard McLaughlin,Franc Kalanderi,Franc Kalanderi,NaN,12/30/2021,Travelers Grp
4,Ashley,Zapotosky,122 Maple Ave.,NJ,07057,ashzap12@gmail.com,(201) 446-1709,WSDC-03663,6BRPG0000007214500,$0.00,Pending,NaN,Linda Sabino,Linda Sabino,Linda Sabino,12/27/2021,Nationwide Corp Grp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36916,Mr,..,43 Fremont St,MA,02145,NaN,(508) 294-8812,WSDC-04497,NaN,NaN,NaN,Joshua Grant,NaN,NaN,NaN,NaN,NaN
36917,Marnie,Lachinsky,3705 Westmark Dr,MN,55345-2246,NaN,NaN,WSDC-04636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36918,Allison,Sorbo,61 Wyandemere Dr,NJ,07677,asorbo80@gmail.com,(551) 247-5454,WSDC-05027,NaN,NaN,NaN,Christie Murphy,NaN,NaN,NaN,NaN,NaN
36919,James,Poltrictzky,775 Washington Rd.,NJ,08859,poltrijp@aol.com,(732) 254-8896,WSDC-05186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
import pickle
import pandas as pd 

In [30]:
processed_cust_num_file = open('1115_2021/processed_cust_num_set.txt', 'wb')
obj = set()
pickle.dump(obj, processed_cust_num_file)

In [31]:
processed_policy_num_file = open('1115_2021/processed_policy_num_set.txt', 'wb')
obj = set()
pickle.dump(obj, processed_policy_num_file)

In [19]:
processed_cust_num_file = open('processed_cust_num_set.txt', 'rb')
objects = []
while True:
    try:
        objects.append(pickle.load(processed_cust_num_file))
    except EOFError:
        break
processed_cust_num_file.close()
processed_cust_num_set = set(objects[1:])

FileNotFoundError: [Errno 2] No such file or directory: 'processed_cust_num_set.txt'

In [15]:
waterstone_policies = pd.read_csv('waterstone_policies.csv', index_col= 0)

In [16]:
final_customers = pd.DataFrame(columns= waterstone_policies.columns)
for index, row in waterstone_policies.iterrows():
    cust_num =  row['Customer Number'].lower() if isinstance(row['Customer Number'], str) else ''
#     if cust_num not in processed_cust_num_set:
#         final_customers = final_customers.append(waterstone_policies.loc[index],  ignore_index = True)
#         processed_cust_num_set.add(cust_num)

In [17]:
final_customers.reindex(columns=[*waterstone_policies.columns.to_list(), 'If Exist'])

,Customer First Name,Customer Last Name,Customer Street Address,Customer State/Province,Customer Zip Code,Customer Email,Customer Phone Number,Customer Number,Policy Number,Policy Premium,...,CSR On Customer,Agent On Policy,CSR On Policy,Producer,Effective Date,Parent Carrier,Writing Carrier,Adjust Carriers,MGA/Brokers,If Exist


In [32]:
final_customers.to_csv('1115_2021/processed_customers_records_11152021.csv')

In [ ]:
import csv

In [31]:
# cnt = 0
# rows = []
# with open('tmp_2020_customer_processed.csv', 'r') as f:
#     reader = csv.DictReader(f)
#     for i, row in enumerate(reader):
#         if i%2 == 1:
#             continue
#         rows.append(row)
# len(rows)

1436

In [43]:
rows = []
with open('customer_processed.csv', 'r') as f:
    reader = csv.reader(f)
    header = next(reader)
    print(header)
    for i, r in enumerate(reader):
        if i %2 == 1:
            continue
        rows.append(r)

['', 'Customer First Name', 'Customer Last Name', 'Customer Street Address', 'Customer State/Province', 'Customer Zip Code', 'Customer Email', 'Customer Phone Number', 'Customer Number', 'Policy Number', 'Policy Premium', 'Policy Status', 'CSR On Customer', 'Agent On Policy', 'CSR On Policy', 'Producer', 'Effective Date', 'Parent Carrier', 'Original Agent', 'Original CSR', 'If Exist']


In [57]:
df = pd.read_csv('customer_processed.csv', index_col=0)

In [18]:
waterstone_policies

,Customer First Name,Customer Last Name,Customer Street Address,Customer State/Province,Customer Zip Code,Customer Email,Customer Phone Number,Customer Number,Policy Number,Policy Premium,Policy Status,CSR On Customer,Agent On Policy,CSR On Policy,Producer,Effective Date,Parent Carrier,Writing Carrier,Adjust Carriers,MGA/Brokers
0,Tony,Gross,P.O. Box 187,NJ,07849,rmgross@verizon.net,(973) 584-6311,WSDC-03825,KKO0000008839000,1399.0,Cancelled,Linda Sabino,Linda Sabino,Linda Sabino,Linda Sabino,2022-06-26 00:00:00,Nationwide Corp Grp,National Cas Co,Nationwide Corp Grp,"k&k insurance group, inc."
1,Tony,Gross,P.O. Box 187,NJ,07849,tonygross1932@gmail.com,(973) 584-6311,WSDC-03825,KKO0000008839000,1399.0,Cancelled,Linda Sabino,Linda Sabino,Linda Sabino,Linda Sabino,2022-06-26 00:00:00,Nationwide Corp Grp,National Cas Co,Nationwide Corp Grp,"k&k insurance group, inc."
2,Ashley,Zapotosky,122 Maple Ave.,NJ,07057,ashzap12@gmail.com,(201) 446-1709,WSDC-03663,6BRPG0000007214500,0.0,Pending,NaN,Linda Sabino,Linda Sabino,Linda Sabino,2021-12-27 00:00:00,Nationwide Corp Grp,National Cas Co,Nationwide Corp Grp,"k&k insurance group, inc."
3,Ashley,Zapotosky,122 Maple Ave.,NJ,07057,info@fitnessinsurance-kk.com,(201) 446-1709,WSDC-03663,6BRPG0000007214500,0.0,Pending,NaN,Linda Sabino,Linda Sabino,Linda Sabino,2021-12-27 00:00:00,Nationwide Corp Grp,National Cas Co,Nationwide Corp Grp,"k&k insurance group, inc."
4,Ashley,Zapotosky,122 Maple Ave.,NJ,07057,tiffany.a.haugh@kandkinsurance.com,(201) 446-1709,WSDC-03663,6BRPG0000007214500,0.0,Pending,NaN,Linda Sabino,Linda Sabino,Linda Sabino,2021-12-27 00:00:00,Nationwide Corp Grp,National Cas Co,Nationwide Corp Grp,"k&k insurance group, inc."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6396,Andrew,Napolitano,873 Broadway,NJ,07104,horshamquestions@hullco.com,(201) 303-8552,WSDC-03358,Pending,0.0,Pending,Linda Sabino,Linda Sabino,NaN,NaN,0001-01-01 00:00:00,No Carrier,No Carrier,No Carrier,NaN
6397,Andrew,Napolitano,873 Broadway,NJ,07104,josie@forlenzacpa.com,(201) 303-8552,WSDC-03358,Pending,0.0,Pending,Linda Sabino,Linda Sabino,NaN,NaN,0001-01-01 00:00:00,No Carrier,No Carrier,No Carrier,NaN
6398,Andrew,Napolitano,873 Broadway,NJ,07104,mschultz@hullco.com,(201) 303-8552,WSDC-03358,Pending,0.0,Pending,Linda Sabino,Linda Sabino,NaN,NaN,0001-01-01 00:00:00,No Carrier,No Carrier,No Carrier,NaN
6399,Andrew,Napolitano,873 Broadway,NJ,07104,pdriscoll@hullco.com,(201) 303-8552,WSDC-03358,Pending,0.0,Pending,Linda Sabino,Linda Sabino,NaN,NaN,0001-01-01 00:00:00,No Carrier,No Carrier,No Carrier,NaN


### Policy Based Marking

#### Build policy dataframe

In [25]:
waterstone_processed_policy_records = pd.DataFrame(columns= ['Customer First Name', 'Customer Last Name', 'Customer Number', 'Policy Num', 'Policy Premium', 'Policy Status', 'Agent On Policy'])

In [33]:
waterstone_processed_policy_records.to_csv('1115_2021/processed_policy_records_11152021.csv')